In [47]:
import requests, bs4, nltk.corpus
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
from time import sleep
# nltk.download('stopwords')
from nltk.corpus import stopwords

In [48]:
# Misc func.

def print_df(df):
    for i in df:
        print(i)

In [49]:
def get_loc(result):
    try:
        return entry.find('div', {'class': 'companyLocation'}).text
    except Exception as e:
        return 'NA'


def get_comp(result):
    try:
        return result.find('span', {'class': 'companyName'}).text
    except:
        return 'NA'


def get_job(result):
    try:
        return result.find('h2', {'class': 'jobTitle'}).text
    except:
        return 'NA'


def get_sal(result):
    try:
        return result.find('div', {'class': 'salaryOnly'}).text
    except:
        return 'NA'


def get_desc(result):
    try:
        return result.find('div', {'class': 'job-snippet'}).text
    except:
        return 'NA'
def get_rating(result):
    try:
        return result.find('span', {'class': 'ratingNumber'}).text
    except:
        return 'NA'

In [50]:
job_titles = [
    'Data+Scientist', 'full+stack+developer', 'big+data+engineer',
    'software+architect', 'blockchain', 'devops+engineer', 'cloud+architect',
    'artificial+intelligence', 'product+manager'
]

len(job_titles)

9

In [51]:
results = []

In [59]:
max_results_per_job = 10000
start_time = datetime.now()
print(start_time)

try:
    for job in job_titles:
        print("---------- Started for " + job)
        for start in range(0, max_results_per_job, 15):
            url = "https://in.indeed.com/jobs?q=" + job + "&start=" + str(
                start)
            html = requests.get(url)
            soup = BeautifulSoup(html.text, 'html.parser')
            for result in soup.find_all('div', {'class': 'job_seen_beacon'}):
                results.append(result)
            sleep(3)
        print("Gathered " + str(len(results)) + " jobs so far")
        print("---------- Completed for " + job)
        print("-------------------------------")
    print("Scraping done !!! Check logs for more info")
except Exception as e:
    print("ERROR: Something went wrong ", e)

end_time = datetime.now()
print(end_time)
print(end_time - start_time)

2022-05-17 03:56:36.742716
---------- Started for Data+Scientist
Gathered 10100 jobs so far
---------- Completed for Data+Scientist
-------------------------------
---------- Started for full+stack+developer
Gathered 20105 jobs so far
---------- Completed for full+stack+developer
-------------------------------
---------- Started for big+data+engineer
Gathered 25880 jobs so far
---------- Completed for big+data+engineer
-------------------------------
---------- Started for software+architect
Gathered 25880 jobs so far
---------- Completed for software+architect
-------------------------------
---------- Started for blockchain


KeyboardInterrupt: 

In [60]:
df_jobs = pd.DataFrame(
    columns=['location', 'title', 'company', 'salary', 'summary', 'rating'])

In [61]:
for entry in results:
    location = get_loc(entry)
    title = get_job(entry)
    company = get_comp(entry)
    salary = get_sal(entry)
    desc = get_desc(entry)
    rating = get_rating(entry)
    df_jobs.loc[len(df_jobs)] = [
        location, title, company, salary, desc, rating
    ]

In [65]:
df_jobs[df_jobs.duplicated()]

,location,title,company,salary,summary,rating
60,"New Delhi, Delhi",Data Scientist,Eoraa & Co.,NA,\nProficient with Python/R.\nStrong communicat...,NA
105,"Chennai, Tamil Nadu",newClinical Data Scientist,Pfizer,NA,\nEnsure operational excellence in collaborati...,4.2
106,"Bengaluru, Karnataka",newData Scientist,Happay,"₹8,00,000 - ₹12,00,000 a year","\n1-2 Years as a data scientist, machine learn...",NA
107,"Bengaluru, Karnataka",Data Scientist,Applied Materials Inc.,"₹5,26,846 - ₹17,85,107 a year","\nAcquire, cleanse, organize and mine massive ...",3.9
108,"Noida, Uttar Pradesh",newSenior Data Scientist,Jorie Healthcare Partners,"₹7,00,000 - ₹15,00,000 a year",\nHas *expertise in implementing complex stati...,NA
...,...,...,...,...,...,...
25875,"Bengaluru, Karnataka",Principal Software Engineer,Intuit,NA,\nCoach and mentor other engineers on methods ...,4.2
25876,"Bengaluru, Karnataka","Software Engineer II, Backend - Technical Privacy",Uber,NA,"\nProcess massive amount of data, build softwa...",3.7
25877,"Pune, Maharashtra",Advisory Quality Assurance Software Engineer,Pitney Bowes,NA,\nAWS Services experience preferably in big da...,3.6
25878,"Hyderabad, Telangana",Systems Engineer- WIT ITSM Tools,Micron,NA,\nExperience with Performance Analytics and ot...,3.7


In [66]:
df_jobs.drop_duplicates(inplace = True, keep='first')
df_jobs
df_jobs.to_csv(r'./Dataset/job_scrap.csv')
# df_job_scrap.describe()

In [43]:
df_jobs = pd.read_csv(r'./Dataset/job_raw.csv')
df_jobs.head()
df_jobs.isna().sum()

Unnamed: 0       0
location         0
title            0
company          0
salary        4102
summary          0
rating        2392
dtype: int64

In [67]:
df_jobs[df_jobs.duplicated()]
df_jobs.drop(columns=['Unnamed: 0'], inplace=True)

KeyError: "['Unnamed: 0'] not found in axis"

In [70]:
display(df_jobs.describe())
print(df_jobs.isna().sum())

,location,title,company,salary,summary,rating
count,2709,2709,2709,960,2709,1355
unique,236,1403,1380,430,2302,29
top,"Bengaluru, Karnataka",Data Scientist,Uber,Full-time,Our vision is to transform how the world uses ...,3.7
freq,733,168,94,282,8,269


location       0
title          0
company        0
salary      1749
summary        0
rating      1354
dtype: int64


In [69]:
df_jobs.replace(to_replace='NA', value=np.nan, inplace=True)

In [72]:
df = df_jobs

In [73]:
print('Unique Locations  \t:', len(df.location.unique()))
print('Unique Title  \t\t:', len(df.title.unique()))
print('Unique Company  \t:', len(df.company.unique()))
print('Unique Salary \t\t:', len(df.salary.unique()))
print('Unique Rating \t\t:', len(df.rating.unique()))
# Unique Locations  	: 385
# Unique Title  		: 3409
# Unique Company  	: 2426
# Unique Salary 		: 605
# Unique Rating 		: 31

Unique Locations  	: 236
Unique Title  		: 1403
Unique Company  	: 1380
Unique Salary 		: 431
Unique Rating 		: 30


In [13]:
# """
# * Noise found
#      1. 'Remote in '
#      2. '+1 location'
#      3. 'Hybrid remote in '
#      4. '+7 location'
#      5. '+2 location'
#      6. 'Temporarily Remote in '
# """

In [74]:
stop_words = stopwords.words('english') + [
    'in', 'remote', 'Remote', 'location', 'locations', 'Temporarily', 'Hybrid',
    'India'
]
df['clean_location'] = df['location'].apply(lambda x: ' '.join(
    [word for word in x.split() if word not in (stop_words)]))

remove_list = ['+1', '+2', '+3', '+4', '+5', '+6', '+7', '+8', '+9', '9']
for i in remove_list:
    df.clean_location = df.clean_location.str.replace(i, '', regex=False)

df.clean_location = df.clean_location.str.strip()

In [75]:
df.head()

,location,title,company,salary,summary,rating,clean_location
0,"Chennai, Tamil Nadu",newClinical Data Scientist,Pfizer,NaN,\nEnsure operational excellence in collaborati...,4.2,"Chennai, Tamil Nadu"
1,"Bengaluru, Karnataka",newData Scientist,Happay,"₹8,00,000 - ₹12,00,000 a year","\n1-2 Years as a data scientist, machine learn...",NaN,"Bengaluru, Karnataka"
2,"Noida, Uttar Pradesh",newSenior Data Scientist,Jorie Healthcare Partners,"₹7,00,000 - ₹15,00,000 a year",\nHas *expertise in implementing complex stati...,NaN,"Noida, Uttar Pradesh"
3,"Bengaluru, Karnataka",Data Scientist,Applied Materials Inc.,"₹5,26,846 - ₹17,85,107 a year","\nAcquire, cleanse, organize and mine massive ...",3.9,"Bengaluru, Karnataka"
4,"Hyderabad, Telangana+1 location",newData & Applied Scientist,Microsoft,NaN,"\nWe are looking for a strong IC, who can lead...",4.2,"Hyderabad, Telangana"


In [76]:
print('Uncleaned Locations  \t:', len(df.location.unique()))
print('Cleaned Locations  \t:', len(df.clean_location.unique()))

Uncleaned Locations  	: 236
Cleaned Locations  	: 126


In [77]:
# print_df(df.title.unique())

In [78]:
stop_words = stopwords.words('english') +  ['Scien...', 'Forecastin...', '-', 'location', 'locations','Temporarily', 'Hybrid', 'India']
df['clean_title'] = df['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [79]:
noise_char_title = ['new', '...', '-', '/', '(', ')', ':', '|', ',']
for i in noise_char_title:
    df.clean_title = df.clean_title.str.replace(i, '', regex=False)

In [80]:
print_df(df.clean_title)

Clinical Data Scientist
Data Scientist
Senior Data Scientist
Data Scientist
Data & Applied Scientist
Senior Data Scientist
Data Scientist
Data Scientist AIML Phython
Data Scientist
Data Scientist
Senior Data scientist
Data Scientist OrgSolutions
Data Scientist
Data ScientistGoFood
Data Scientist
Data Scientist
Data Scientist Analyst
Data Scientist
Data Scientist
Deep Learning Specialist Data Scientist
Data Scientist
Data Scientist  Analyst
Associate Data Scientist Analytical
Sr. Data Scientist Deep Learning
Data Scientist Operation Research
Head Data Risk
Data Scientist
Data Scientist
Data Scientist
Data Scientist V
Data ScientistGeospatial Analytics
Data Scientist
Data Scientist
Data Scientist
Data Scientist Pricing Analytics
Data Scientist
Data Scientist Enablement
Data Scientist
Data Scientist
Data Scientist Advanced Analytics
Data Scientist
Data Scientist Clinical Trial
Data Scientist
Applied Scientist
Data Scientist
Data Scientist
Data Scientist
Data Scientist Noida
Analyst Data S

Full Stack Developer
Java Full Stack Developer
UIUX Developer
ETL Developer
Full Stack Developer
Tech Lead
Senior Full Stack Cloud Developer
Python Full Stack Developer
Full Stack Developer MEANMERN
Software Developer
Java Full stack Developer
Full Stack Developer
React Developer
Full Stack Developer
Full Stack Developer
Java Full Stack Developer
Full Stack Developer
Full Stack Developer
Full Stack Developer
BI Developer SQL
Full Stack Developer
Live Project Internship
Software Engineer Full Stack
Project Development Manager
Sr.Net Developer
C#  .NET SQL Server HTML CSS JavaScript jQuery Bootst
Full Stack Developer
ReactJS Developer
JAVA Developer
Full Stack Developer Nodejs Reactjs PHP Angularjs Full
MEAN full stack Developer
PHP  Full Stack Developer
Python Developer
Dot Net Developer
Node js developer
Sr. Full Stack Developer
Sr. Node JS Developer
REMOTE Senior Full Stack Developer
Full Stack Developer
Junior Full Stack Developer React
Dotnet Developer
Fullstack Engineer E3661
K2 BP

Big Data Engineer
Data Engineer II
Ecosystem Junior Data Engineer
Senior EngineerBig Data
Data Engineer
MBOS_Data Science Engineer Image Database
Senior Consultant Data Engineer
Data Engineer ETL Data Warehouse
Senior Data Engineer
Cloud Data Engineer
Big Data Engineer
Senior Big Data Engineer
Data Engineer
Big Data Engineer
Azure Data Engineer
Big Data Lead Engineer
Data Engineer
Data Engineer DIDP
Data Engineer
Senior Data Engineer
Data Engineer
Software Engineer
Senior Data Engineer
Big Data Engineer
Big data Engineer
Engineer
Data Engineer Matlab Python
Data Engineer III
Staff Data Engineer
Data Engineer
Senior Data Engineer
Data Engineer
Big Data Engineer
Azure Data Engineer
Engineer Software QA
Data Engineer
Big Data Engineer_Associate_C2MA
Senior QA Engineer Big Data
Data Scientist
Senior Data Engineer
Data Engineer
Associate Data Engineer
Data Engineer_Associate_CBGT_C2MA
Senior Big Data Engineer
Big Data Engineer
Data Engineer Data Warehouse
Data Migration Analyst
Data Enginee

Senior Software Engineer – Java Big Data
Sr. Data Engineer
Software Engineer  Backend  35years
Implementation Engineer
DevOps Engineering EngineerII
Data Architecture Engineer
Java Backend Engineer Data Merchant Engineering
Cloud Data Engineer
Data Engineer
Data Engineer
QA Automation Engineer Intern
Machine Learning Engineer
Lead Consultant ATM Senior Data Engineer
Agile Engineer
MACHINE LEARNING DATA ENGINEER
Associate Software Engineer
Data Scientist & ML Engineer Lead
Staff Engineer ETX
Big Data Tableau Developer Assistant Vice President
Assistant Manager_Senior Engineer_Big data Developer_Pune
Senior Technical Lead Data Engineer
Big Data Engineer
Senior Splunk SIEM Engineer
Technical Services Engineer – Guardium L2
Business Intelligence Engineer Alexa Skills Kit
Data Engineer WFS ACCS Planning
Machine Learning Engineer
Big Data Lead
Data Scientist
Data Engineer III GBS IND
Software Engineer II Data Science
Engineer 2 Engineering Operations
Site Reliability Developer 4 2022P
Manage

In [21]:
for i in results:
    print(get_job(i))

NameError: name 'results' is not defined

In [81]:
df['clean_title']

0                      Clinical Data Scientist
1                               Data Scientist
2                        Senior Data Scientist
3                               Data Scientist
4                     Data & Applied Scientist
                         ...                  
25242                       Sr. Data Scientist
25243                Privacy Software Engineer
25265                         Data Scientist I
25283    Software Engineer Data Engineering II
25655               Manager Data Engineer Lead
Name: clean_title, Length: 2709, dtype: object

In [82]:
df_comp_rat = df_jobs[['company', 'rating']]

In [83]:
df_comp_rat.drop_duplicates(keep='first')

,company,rating
0,Pfizer,4.2
1,Happay,NaN
2,Jorie Healthcare Partners,NaN
3,Applied Materials Inc.,3.9
4,Microsoft,4.2
...,...,...
20847,Rakuten India Enterprises Private Limited,3.4
24060,BrightTALK,NaN
25119,Blackhawk Network,3.4
25145,Pratiti Technologies,NaN


In [84]:
import pandasql as psql

In [85]:
df_temp = psql.sqldf("SELECT DISTINCT(clean_location) FROM df")

In [86]:
df_clean_loc_list = df_temp.clean_location.str.split(',')
for i in df_clean_loc_list:
    if len(i) == 1:
        print(i)

['']
['Delhi']
['Orissa']
['India']
['Maharashtra']
['Tamil Nadu']
['Telangana']
['West Bengal']
['locationRemote']
['Kerala']
['Kalyananagar']
['Goa']
['locationsRemote']


In [87]:
df_sal_list = df_jobs.salary.str.split(' ')
for i in df_sal_list:
    print(i)

nan
['₹8,00,000', '-', '₹12,00,000', 'a', 'year']
['₹7,00,000', '-', '₹15,00,000', 'a', 'year']
['₹5,26,846', '-', '₹17,85,107', 'a', 'year']
nan
nan
['₹10,00,000', 'a', 'year']
nan
nan
['₹5,00,000', '-', '₹7,00,000', 'a', 'year']
nan
nan
nan
nan
nan
nan
nan
['₹10,00,000', '-', '₹12,00,000', 'a', 'year']
nan
nan
nan
nan
nan
nan
['₹13,00,000', '-', '₹24,00,000', 'a', 'year']
nan
['₹4,10,983', '-', '₹15,42,344', 'a', 'year']
['₹30,006', '-', '₹84,211', 'a', 'month']
['₹20,00,000', 'a', 'year']
nan
nan
nan
['₹5,00,000', '-', '₹15,00,000', 'a', 'year']
nan
['₹13,00,000', '-', '₹24,00,000', 'a', 'year']
nan
nan
nan
nan
nan
nan
nan
nan
nan
['₹2,96,790', '-', '₹14,75,921', 'a', 'year']
nan
nan
['₹17,00,000', 'a', 'year']
nan
nan
nan
['₹3,61,402', '-', '₹15,27,932', 'a', 'year']
nan
nan
['₹35,000', 'a', 'month']
nan
nan
nan
nan
nan
nan
nan
['₹10,00,000', '-', '₹12,00,000', 'a', 'year']
['₹5,000', '-', '₹15,000', 'a', 'month']
nan
['₹1,00,000', '-', '₹1,20,000', 'a', 'year']
nan
['₹12,00,000', 

['₹80,000', '-', '₹1,00,000', 'a', 'month']
['₹4,00,000', '-', '₹15,00,000', 'a', 'year']
['₹4,00,000', '-', '₹7,00,000', 'a', 'year']
['₹25,000', '-', '₹60,000', 'a', 'month']
nan
nan
nan
nan
['₹5,00,000', 'a', 'year']
['₹6,00,000', '-', '₹15,00,000', 'a', 'year']
nan
nan
['₹14,00,000', '-', '₹16,00,000', 'a', 'year']
nan
['₹20,000', 'a', 'month']
['₹13,00,000', 'a', 'year']
['₹30,000', '-', '₹50,000', 'a', 'month']
['₹20,000', '-', '₹60,000', 'a', 'month']
['₹15,000', 'a', 'month']
nan
nan
nan
nan
nan
['₹2,88,747', '-', '₹12,74,756', 'a', 'year']
['₹3,00,000', '-', '₹15,00,000', 'a', 'year']
['₹60,000', '-', '₹70,000', 'a', 'month']
['₹7,32,243', '-', '₹13,75,130', 'a', 'year']
nan
['₹8,00,000', '-', '₹15,00,000', 'a', 'year']
nan
['₹4,50,000', '-', '₹6,00,000', 'a', 'year']
nan
['₹2,40,000', 'a', 'year']
nan
nan
['₹4,00,000', '-', '₹12,00,000', 'a', 'year']
nan
nan
nan
['₹16,16,768', 'a', 'year']
['₹1,00,000', '-', '₹1,20,000', 'a', 'month']
['₹20,000', '-', '₹40,000', 'a', 'month']

['₹15,00,000', 'a', 'year']
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
['₹6,00,000', '-', '₹20,00,000', 'a', 'year']
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
['₹15,00,000', 'a', 'year']
['₹44,43,829', '-', '₹60,17,875', 'a', 'year']
nan
nan
nan
nan
nan
['₹10,00,000', '-', '₹12,00,000', 'a', 'year']
nan
nan
['₹9,00,000', '-', '₹12,00,000', 'a', 'year']
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
['₹7,00,000', 'a', 'year']
nan
nan
nan
nan
nan
['₹15,00,000', '-', '₹18,00,000', 'a', 'year']
nan
nan
nan
['Full-time']
['Full-time']
nan
['Full-timeFlexible', 'shift', '+2']
['₹15,00,000', '-', '₹25,00,000', 'a', 'yearFull-timeDay', 'shift']
nan
['Full-time']
nan
['Full-time']
['₹1,00,000', '-', '₹1,20,000', 'a', 'year']
['Full-time']
nan
['₹12,00,000', '-', '₹22,00,000', 'a', 'year']
['Full-time']
['Full-time']
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
['Full-time']
nan
nan
['Full-time']
n

In [88]:
df_jobs

,location,title,company,salary,summary,rating,clean_location,clean_title
0,"Chennai, Tamil Nadu",newClinical Data Scientist,Pfizer,NaN,\nEnsure operational excellence in collaborati...,4.2,"Chennai, Tamil Nadu",Clinical Data Scientist
1,"Bengaluru, Karnataka",newData Scientist,Happay,"₹8,00,000 - ₹12,00,000 a year","\n1-2 Years as a data scientist, machine learn...",NaN,"Bengaluru, Karnataka",Data Scientist
2,"Noida, Uttar Pradesh",newSenior Data Scientist,Jorie Healthcare Partners,"₹7,00,000 - ₹15,00,000 a year",\nHas *expertise in implementing complex stati...,NaN,"Noida, Uttar Pradesh",Senior Data Scientist
3,"Bengaluru, Karnataka",Data Scientist,Applied Materials Inc.,"₹5,26,846 - ₹17,85,107 a year","\nAcquire, cleanse, organize and mine massive ...",3.9,"Bengaluru, Karnataka",Data Scientist
4,"Hyderabad, Telangana+1 location",newData & Applied Scientist,Microsoft,NaN,"\nWe are looking for a strong IC, who can lead...",4.2,"Hyderabad, Telangana",Data & Applied Scientist
...,...,...,...,...,...,...,...,...
25242,"Bengaluru, Karnataka",Sr. Data Scientist,Blackhawk Network,Full-time,\n10+ years of overall professional experience...,3.4,"Bengaluru, Karnataka",Sr. Data Scientist
25243,"Hyderabad, Telangana",Privacy Software Engineer,Microsoft,Full-time,\nExperience with telemetry and data mining an...,4.2,"Hyderabad, Telangana",Privacy Software Engineer
25265,"Bengaluru, Karnataka",Data Scientist - I,ADCI - Karnataka,NaN,\nDevelop efficient data querying infrastructu...,3.5,"Bengaluru, Karnataka",Data Scientist I
25283,"Hyderabad, Telangana+1 location",Software Engineer in Data Engineering - II,Uber,Full-time,\nThis role also designs and develops large-sc...,3.7,"Hyderabad, Telangana",Software Engineer Data Engineering II
